In [1]:
import torch
import torch.optim as optim
import numpy as np
import os
import argparse
import time, datetime
import matplotlib; matplotlib.use('Agg')
from src import config, data
from src.checkpoints import CheckpointIO
from collections import defaultdict
import shutil
from tensorboardX import SummaryWriter
from src.utils.libkdtree import KDTree
# import pickle
# from scipy.spatial import cKDTree

In [2]:
!which cuda

In [3]:
cfg = config.load_config('configs/pointcloud/grid.yaml', 'configs/default.yaml')
is_cuda = (torch.cuda.is_available())
device = torch.device("cuda" if is_cuda else "cpu")

In [10]:
train_dataset = config.get_dataset('train', cfg)
shape = train_dataset[0]
shape['inputs.kdtree_points'].shape

(100000, 3)

In [8]:
shape.keys()

dict_keys(['points', 'points.occ', 'points.dist', 'points.closest_points', 'inputs', 'inputs.normals', 'inputs.kdtree_points'])

In [26]:
dast.shape

(5, 100)

In [5]:
s = pickle.dumps(b)

NameError: name 'b' is not defined

In [5]:
def collate(data_list):
    return {k:[x[k] for x in data_list] for k in data_list[0].keys()}

In [11]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, num_workers=8, shuffle=True,
    collate_fn=data.collate_remove_none,
    worker_init_fn=data.worker_init_fn)

In [12]:
points_k1 = torch.rand(32,2048,3)

In [13]:
batch = next(train_loader.__iter__())

In [14]:
batch['inputs.kdtree_points'].shape

torch.Size([32, 100000, 3])

In [40]:

#distance_matrix(torch.rand(16, 100000, 3), torch.rand(16, 2048, 3)).topk(1)
points_k1_cpu = points_k1.detach().cpu().numpy()
points_kdtree = batch['inputs.kdtree_points'].cpu().numpy()
d_k1 = torch.tensor([KDTree(points_kdtree[i]).query(x)[0] for i, x in enumerate(points_k1_cpu)])


In [41]:
d_k1.shape

torch.Size([32, 2048])

In [2]:
def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.
    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist


In [28]:
a = torch.rand(32, 2048, 3, requires_grad=True).cuda()

In [29]:
b = torch.rand(32, 50000, 3).cuda()

In [30]:
square_distance(a, b).topk(10).values.shape

torch.Size([32, 2048, 10])

In [31]:
torch.cuda.empty_cache()
import gc; gc.collect()

0

In [28]:
surface_points =  shape.get('inputs')
query_points = shape.get('points')
query_points.shape

(2048, 3)

In [31]:
kdtree = KDTree(surface_points)
dist, idx = kdtree.query(query_points)

In [61]:
idx.shape
surface_points[idx].shape
idx.max()

2998

In [38]:
ls data/ShapeNet/02691156/d171967c6353177bb87697d3904b168b/

img_choy2016/  model.binvox  pointcloud.npz  points.npz


In [130]:
def distance(points_file, pointcloud_file, name, sigma):
    pointcloud = pointcloud_file.get('points')
    points = pointcloud + sigma*np.random.randn(pointcloud.shape) #points_file.get('points')
    kdtree = KDTree(pointcloud)
    dist, idx = kdtree.query(points)
    np.savez(name, 
             points=points, occupancies=points_file.get('occupancies'),
             loc=points_file.get('loc'), scale=points_file.get('scale'), dist=dist, closest_points=pointcloud[idx])

In [131]:
from glob import glob
import multiprocessing as mp
from multiprocessing import Pool

In [118]:
sorted(glob('data/ShapeNet/*/*'))[-1]

'data/ShapeNet/04530566/val.lst'

In [132]:
#points_distance is the one with pointclouds coordinates directly, no need to load pointcloud twice during training 
def process(path, sigma):
    points_file = np.load(path +'/points.npz')
    pointcloud_file = np.load(path +'/pointcloud.npz')
    name = path + f'/points_distance_{sigma}.npz'
    distance(points_file, pointcloud_file, name)

In [104]:
process(glob('data/ShapeNet/*/*')[0])

(100000,)


In [133]:
#p = Pool(mp.cpu_count())

In [134]:
#m = p.map(process, glob('data/ShapeNet/*/*/')); p.close(); p.join()

In [146]:
test = np.load('data/ShapeNet/02691156/d171967c6353177bb87697d3904b168b/pointcloud.npz')

In [147]:
list(test.keys())

['points', 'normals', 'loc', 'scale']

In [145]:
test['points'].shape

(100000, 3)

In [148]:
def distance(points, kdtree):
    dist, idx = kdtree.query(points)
    return dist

In [3]:
pwd

'/home/nizar/convolutional_occupancy_networks'

In [1]:
import torch 

In [2]:
cd ChamferDistancePytorch

/home/nizar/convolutional_occupancy_networks/ChamferDistancePytorch


In [4]:
import torch 
import chamfer3D.dist_chamfer_3D
#chamLoss(torch.rand(2,1, 3).cuda(), torch.rand(2,1, 3).cuda())

In [ ]:
chamLoss = chamfer3D.dist_chamfer_3D.chamfer_3DDist()
points1 = torch.rand(2, 10, 3).cuda()
points2 = torch.rand(2, 20, 3, requires_grad=True).cuda()
dist1, dist2, idx1, idx2 = chamLoss(points1, points2)

In [5]:
chamLoss = chamfer3D.dist_chamfer_3D.chamfer_3DDist()

In [ ]:
chamLoss(torch.rand(2,1, 3).cuda(), torch.rand(2,1, 3).cuda())

In [5]:
import torch
print(torch.__version__)
print(torch.cuda.get_arch_list())

1.9.0+cu111
['sm_37', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'sm_80', 'sm_86']
